## Model Comparision 
Comparision of YOLO models (yoloV8m, yoloV8n, yoloV5n, yolov11n) on GateDetection Dataset (https://www.kaggle.com/datasets/mobilal14/sauvc-dataset) on Tesla P100-PCIE-16GB GPU

In [2]:
import os
import csv
import cv2
import pandas as pd
from ultralytics import YOLO
import matplotlib.pyplot as plt

In [1]:
MODELS_DIR = "/Users/mohammadbilal/Documents/Projects/GateDetection/Models"
DATA_YAML = "/Users/mohammadbilal/Documents/Projects/GateDetection/Dataset/data.yaml"
TEST_IMAGES = "/Users/mohammadbilal/Documents/Projects/GateDetection/Dataset/test"
OUTPUT_DIR = "/Users/mohammadbilal/Documents/Projects/GateDetection/output"
CONF = 0.6
IOU = 0.5
IMG_SIZE = 640
DEVICE = "cpu"   

In [3]:
VIS_DIR = os.path.join(OUTPUT_DIR, "visuals")
os.makedirs(VIS_DIR, exist_ok=True)


results_table = []

model_files = [f for f in os.listdir(MODELS_DIR) if f.endswith(".pt")]

print(f"Evaluating {len(model_files)} models...")
for i, file in enumerate(model_files):
    file = file.replace(".pt", "")
    print(f"Model - {i} : {file}")

Evaluating 4 models...
Model - 0 : yolo_v11n
Model - 1 : yolo_v8n
Model - 2 : yolo_v5n
Model - 3 : yolo_v8m


In [ ]:
for model_file in model_files:
    model_path = os.path.join(MODELS_DIR, model_file)
    model_name = model_file

    model = YOLO(model_path)

    metrics = model.val(
        data=DATA_YAML,
        imgsz=IMG_SIZE,
        conf=CONF,
        iou=IOU,
        device=DEVICE,
        plots=True,
        save_json=True,
        project=OUTPUT_DIR,
        name=model_name
    )

    results_table.append({
        "model": model_name,
        "mAP@0.5": metrics.box.map50,
        "mAP@0.5:0.95": metrics.box.map,
        "precision": metrics.box.mp,
        "recall": metrics.box.mr
    })

    # ---------- VISUAL DIFFS ----------
    model_vis_dir = os.path.join(VIS_DIR, model_name)
    os.makedirs(model_vis_dir, exist_ok=True)

    for i, img_name in enumerate(sorted(os.listdir(TEST_IMAGES))):
        if i >= 50:
            break

        img_path = os.path.join(TEST_IMAGES, img_name)
        img = cv2.imread(img_path)

        if img is None:
            continue

        preds = model(
            img,
            conf=CONF,
            imgsz=IMG_SIZE,
            device=DEVICE
        )

        annotated = preds[0].plot()

        name, _ = os.path.splitext(img_name)
        save_path = os.path.join(model_vis_dir, f"{name}.jpg")

        cv2.imwrite(save_path, annotated)

# ---------------- SAVE CSV ---------------- #
df = pd.DataFrame(results_table)
csv_path = os.path.join(OUTPUT_DIR, "model_comparison.csv")
df.to_csv(csv_path, index=False)

print("\nEvaluation complete.")
print(df)
